In [16]:
import pandas as pd
import coingecko_utils as cg
import numpy as np
import plotly.express as px

rolling_periods = [30,90,180,365]

In [17]:
op_price = cg.get_daily_token_data('optimism')[['date','prices','total_volumes','symbol']]
# op_price = op_price.fillna(0)
prices_cols = []
# display(op_price)
# print( op_price.dtypes )
for i in rolling_periods:
    pcol = 'prices_'+str(i)+'dma'
    vwapcol = 'prices_'+str(i)+'d_vwap'
    op_price[pcol] = op_price[['date','symbol','prices']]\
                                    .groupby(['symbol'])['prices']\
                                    .transform(lambda x: x.rolling(i, min_periods=1).mean() )
    
    vol_rolling_sum = op_price['total_volumes'].rolling(i, min_periods=1).sum()

    op_price[vwapcol] = (op_price['prices'] * op_price['total_volumes']).rolling(i, min_periods=1).sum() / vol_rolling_sum
    
    prices_cols.append(pcol)
    prices_cols.append(vwapcol)

op_price = op_price.sort_values(by='date',ascending=False)
display(op_price)
print(prices_cols)

,date,prices,symbol,total_volumes,prices_30dma,prices_30d_vwap,prices_90dma,prices_90d_vwap,prices_180dma,prices_180d_vwap,prices_365dma,prices_365d_vwap
282,2023-03-08 13:10:38,2.234998,op,3.208215e+08,2.627079,2.687603,1.897787,2.339368,1.426801,1.927988,1.277493,1.684459
281,2023-03-08 00:00:00,2.403660,op,2.900101e+08,2.649580,2.708475,1.884975,2.336855,1.422225,1.919743,1.274098,1.681144
280,2023-03-07 00:00:00,2.515031,op,2.175011e+08,2.667489,2.724138,1.869624,2.332148,1.415429,1.912179,1.270078,1.677189
279,2023-03-06 00:00:00,2.430092,op,2.173004e+08,2.683575,2.736227,1.853445,2.327909,1.407976,1.904413,1.265632,1.673736
278,2023-03-05 00:00:00,2.368444,op,2.004123e+08,2.706017,2.761519,1.837881,2.322839,1.400607,1.893875,1.261458,1.670608
...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-06-04 00:00:00,1.379352,op,4.252324e+08,1.391725,1.382362,1.391725,1.382362,1.391725,1.382362,1.391725,1.382362
3,2022-06-03 00:00:00,1.544824,op,6.528499e+08,1.394819,1.383172,1.394819,1.383172,1.394819,1.383172,1.394819,1.383172
2,2022-06-02 00:00:00,1.237603,op,7.441554e+08,1.344817,1.269347,1.344817,1.269347,1.344817,1.269347,1.344817,1.269347
1,2022-06-01 00:00:00,1.398424,op,9.150572e+07,1.398424,1.398424,1.398424,1.398424,1.398424,1.398424,1.398424,1.398424


['prices_30dma', 'prices_30d_vwap', 'prices_90dma', 'prices_90d_vwap', 'prices_180dma', 'prices_180d_vwap', 'prices_365dma', 'prices_365d_vwap']


In [18]:
fig = px.line(op_price, x="date", y=prices_cols, title='OP Price - Moving Averages (Source: Coingecko)')
fig.show()